
# SHAP Quicklook — Readmission Risk Model

This notebook trains a quick **XGBoost** model for 30‑day readmission, then generates **SHAP** explanations.
- **Dataset path:** `../data/sample_readmissions.csv`
- **Artifacts:** saved to `../artifacts/`


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
import shap
import joblib
from pathlib import Path

DATA_PATH = Path("../data/sample_readmissions.csv")
ARTIFACTS = Path("../artifacts")
ARTIFACTS.mkdir(exist_ok=True, parents=True)

NUMERIC = ["age","length_of_stay","prior_admissions_1y","num_medications","med_adherence_score","avg_systolic_bp","avg_glucose"]
BINARY  = ["chronic_diabetes","chronic_hf","chronic_ckd"]
CATEGORICAL = ["sex","discharge_to","insurance_type"]
FEATURES = NUMERIC + BINARY + CATEGORICAL
TARGET = "readmit_30d"

df = pd.read_csv(DATA_PATH)
X, y = df[FEATURES].copy(), df[TARGET].astype(int).copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

num_pipe = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
bin_pipe = Pipeline([("imputer", SimpleImputer(strategy="most_frequent"))])
cat_pipe = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(handle_unknown="ignore"))])

pre = ColumnTransformer([("num", num_pipe, NUMERIC), ("bin", bin_pipe, BINARY), ("cat", cat_pipe, CATEGORICAL)])
Xtr_p = pre.fit_transform(X_train); Xte_p = pre.transform(X_test)

ros = RandomOverSampler(random_state=42)
Xtr_bal, ytr_bal = ros.fit_resample(Xtr_p, y_train)

model = XGBClassifier(
    n_estimators=400, learning_rate=0.05, max_depth=5,
    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
    random_state=42, tree_method="hist", eval_metric="logloss"
)
model.fit(Xtr_bal, ytr_bal)

proba = model.predict_proba(Xte_p)[:,1]
preds = (proba >= 0.5).astype(int)
auc = roc_auc_score(y_test, proba)
ap  = average_precision_score(y_test, proba)
f1  = f1_score(y_test, preds)
print(f"ROC-AUC: {auc:.3f} | AvgPrec: {ap:.3f} | F1: {f1:.3f}")

joblib.dump(pre, ARTIFACTS / "sklearn_pipeline.pkl")
joblib.dump(model, ARTIFACTS / "model_xgb.pkl")


In [ ]:

# SHAP summary
pre = joblib.load(ARTIFACTS / "sklearn_pipeline.pkl")
model = joblib.load(ARTIFACTS / "model_xgb.pkl")

X_trans = pre.transform(X)
# Sample for performance
background = shap.sample(X_trans, 200)
explainer = shap.TreeExplainer(model)
sv = explainer(background)

plt.figure()
shap.plots.bar(sv, show=False, max_display=15)
plt.tight_layout()
png_path = ARTIFACTS / "shap_summary.png"
plt.savefig(png_path, dpi=160)
print(f"Saved SHAP summary -> {png_path}")
plt.show()


In [ ]:

# Explain a single patient (waterfall)
i = 0  # change index to inspect other patients
x_one = pre.transform(X.iloc[[i]])
sv_one = shap.Explanation(values=explainer.shap_values(x_one)[0], base_values=explainer.expected_value, data=x_one[0])
shap.plots.waterfall(sv_one, max_display=15)
